In [1]:
import os

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'

os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [3]:
def get_timespan(df, num_days, max_days,future_date):
    feature = []
    for i in range(max_days,len(df)-future_date):
        temp_sum = 0
        for j in range(i-num_days,i):
            temp_sum+=df['Price'][j]
        feature.append(temp_sum/num_days)
    return np.array(feature)

def get_dow(df,num_days,max_days,future_date):
    feature = []
    for i in range(max_days, len(df) - future_date):
        temp_sum = 0
        for j in range(i - num_days*7, i,7):
            temp_sum += df['Price'][j-5] + df['Price'][j-6] + df['Price'][j-4]
        feature.append(temp_sum / (num_days*3))
    return np.array(feature)

In [12]:
df = pd.read_csv('a.csv')
X = pd.DataFrame({
    "day_1": get_timespan(df,1,140,30),
    "mean_3": get_timespan(df,3,140,30),
    "mean_7": get_timespan(df,7,140,30),
    "mean_14": get_timespan(df,14,140,30),
    "mean_30": get_timespan(df,30,140,30),
    "mean_60": get_timespan(df,60,140,30),
    "mean_140": get_timespan(df,140,140,30),
    "dow_4": get_dow(df,num_days,max_days,future_date)})
y = df['Price'][170:]
X.head()

NameError: name 'num_days' is not defined

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
t_err = 0
min_err = 10000
best_params = []
for eta in [.1,.3,.5,.7,.9]:
    for mdepth in [2,3,5]:
        for min_cweight in [15,20,25,30,50,60]:            
            param = {}
            param['objective'] = 'reg:linear'
            param['eta'] = eta
            param['max_depth'] = mdepth
            param['silent'] = 1
            param['eval_metric'] = 'rmse'
            param['min_child_weight'] = min_cweight
            param['subsample'] = 0.8
            param['colsample_bytree'] = 0.7
            param['seed'] = 137
            plst = list(param.items())
            temp = 0
            temp2 = 0
            num_rounds = 100
            for curr_fold in range(500,901,100):
                dtrain = xgb.DMatrix(X[:curr_fold],label=y[:curr_fold])
                dval = xgb.DMatrix(X[len(X)-curr_fold:len(X)-curr_fold+150], label=y[len(y)-curr_fold:len(y)-curr_fold+150])
                watchlist = [(dtrain,'train'),(dval,'val')]
                model = xgb.train(plst,dtrain,num_rounds,watchlist,early_stopping_rounds=10,verbose_eval=50)
                val_pred = model.predict(dval)
                temp += mean_squared_error(val_pred,y[len(y)-curr_fold:len(y)-curr_fold+150])
                temp2 += mean_squared_error(y[:curr_fold],model.predict(dtrain))
            
            if min_err > temp:
                min_err = temp/len(list(range(500,901,100)))
                t_err = temp2/len(list(range(500,901,100)))
                best_params = param

print("Validation rmse:", sqrt(min_err))
print("Training rmse:", sqrt(t_err))
print(best_params)

[0]	train-rmse:57.2177	val-rmse:62.4012
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:2.78819	val-rmse:8.16929
Stopping. Best iteration:
[83]	train-rmse:2.18453	val-rmse:7.58594

[0]	train-rmse:58.2016	val-rmse:58.9173
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:2.71504	val-rmse:5.84736
Stopping. Best iteration:
[64]	train-rmse:2.45646	val-rmse:5.7491

[0]	train-rmse:58.1643	val-rmse:61.7933
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:2.75747	val-rmse:2.39912
[99]	train-rmse:2.07061	val-rmse:1.66985
[0]	train-rmse:58.6997	val-rmse:58.6893
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't 

[0]	train-rmse:57.2177	val-rmse:62.4012
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:2.05101	val-rmse:7.59998
Stopping. Best iteration:
[77]	train-rmse:1.6114	val-rmse:7.07033

[0]	train-rmse:58.2016	val-rmse:58.9173
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.97572	val-rmse:6.15942
Stopping. Best iteration:
[65]	train-rmse:1.69314	val-rmse:6.09826

[0]	train-rmse:58.1643	val-rmse:61.7933
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:2.09496	val-rmse:1.64647
[99]	train-rmse:1.3568	val-rmse:0.983508
[0]	train-rmse:58.6997	val-rmse:58.6893
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't 

[0]	train-rmse:57.2177	val-rmse:62.4012
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.40851	val-rmse:7.37155
Stopping. Best iteration:
[75]	train-rmse:1.14823	val-rmse:6.9967

[0]	train-rmse:58.2016	val-rmse:58.9173
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.39769	val-rmse:5.26739
Stopping. Best iteration:
[60]	train-rmse:1.23536	val-rmse:5.21783

[0]	train-rmse:58.1643	val-rmse:61.7933
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.26619	val-rmse:0.950749
[99]	train-rmse:0.8735	val-rmse:0.598319
[0]	train-rmse:58.6997	val-rmse:58.6893
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't

[99]	train-rmse:2.23811	val-rmse:1.46355
[0]	train-rmse:44.7678	val-rmse:50.5416
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[33]	train-rmse:2.04202	val-rmse:8.19397

[0]	train-rmse:45.4909	val-rmse:46.9124
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[20]	train-rmse:2.47309	val-rmse:5.70503

[0]	train-rmse:45.4515	val-rmse:48.4056
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.79644	val-rmse:1.40562
[99]	train-rmse:1.25318	val-rmse:0.95532
[0]	train-rmse:45.8881	val-rmse:46.1274
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1

Will train until val-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[18]	train-rmse:1.82275	val-rmse:6.24699

[0]	train-rmse:45.4515	val-rmse:48.4056
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.16787	val-rmse:0.818456
[99]	train-rmse:0.853321	val-rmse:0.617997
[0]	train-rmse:45.8881	val-rmse:46.1274
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.12074	val-rmse:1.09177
[99]	train-rmse:0.827065	val-rmse:0.815232
[0]	train-rmse:45.6098	val-rmse:42.6985
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.29641	val-rmse:1.19362
[99]	train-rmse:0.965834	val-rmse:0.859627
[0]	train-rmse:44.7678	val-rmse:50.5416
Multiple eval metrics have been passed: 

...

[0]	train-rmse:32.924	val-rmse:30.8154
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.78803	val-rmse:1.54885
[99]	train-rmse:1.27181	val-rmse:1.08236
[0]	train-rmse:32.3966	val-rmse:38.6941
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[11]	train-rmse:2.83073	val-rmse:6.18574

[0]	train-rmse:32.8192	val-rmse:35.0052
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[8]	train-rmse:2.84502	val-rmse:6.75477

[0]	train-rmse:32.798	val-rmse:35.0302
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.36654	val-rmse:0.901438
Stopping. Best iter

Stopping. Best iteration:
[15]	train-rmse:1.87692	val-rmse:4.97265

[0]	train-rmse:32.8192	val-rmse:35.0052
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[9]	train-rmse:2.3018	val-rmse:6.2406

[0]	train-rmse:32.798	val-rmse:35.0302
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.10524	val-rmse:0.707891
[99]	train-rmse:0.79069	val-rmse:0.55941
[0]	train-rmse:33.1301	val-rmse:33.5892
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.09438	val-rmse:0.943451
[99]	train-rmse:0.784015	val-rmse:0.672123
[0]	train-rmse:32.924	val-rmse:30.8154
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't 

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.842224	val-rmse:0.639355
[99]	train-rmse:0.542447	val-rmse:0.406762
[0]	train-rmse:33.1301	val-rmse:33.5892
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.758654	val-rmse:0.731096
[99]	train-rmse:0.523816	val-rmse:0.499535
[0]	train-rmse:32.924	val-rmse:30.8154
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.899449	val-rmse:0.786927
[99]	train-rmse:0.575773	val-rmse:0.514339
[0]	train-rmse:32.3966	val-rmse:38.6941
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[13]	train-rmse:1.55896	val-rmse:8.09721

[0]	train-rmse:32.8192	val-rmse:35.0052
Multiple eval metrics have been pass

Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.63648	val-rmse:1.54813
[99]	train-rmse:1.21167	val-rmse:1.18068
[0]	train-rmse:20.2486	val-rmse:26.8744
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[5]	train-rmse:3.38945	val-rmse:6.47898

[0]	train-rmse:20.2598	val-rmse:23.3455
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[3]	train-rmse:3.46204	val-rmse:5.82563

[0]	train-rmse:20.315	val-rmse:21.6902
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.47825	val-rmse:1.20054
[99]	train-rmse:1.12313	val-rmse:0.838086
[0]	train-rmse:20.

Will train until val-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[5]	train-rmse:3.04358	val-rmse:5.4657

[0]	train-rmse:20.2598	val-rmse:23.3455
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[4]	train-rmse:2.76958	val-rmse:6.10291

[0]	train-rmse:20.315	val-rmse:21.6902
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.06066	val-rmse:0.753497
[99]	train-rmse:0.791939	val-rmse:0.588668
[0]	train-rmse:20.5253	val-rmse:21.1165
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.08983	val-rmse:0.881924
Stopping. Best iteration:
[64]	train-rmse:1.00614	val-rmse:0.837588

[0]	train-rmse:20.3983	val-rmse:18.9845
Multiple eval metrics have been p

In [9]:
plst = list(best_params.items())
dtrain = xgb.DMatrix(X[:len(x)],label=y[:curr_fold])
dval = xgb.DMatrix(X[:len(X)-150], label=y[:len(y)-150])
watchlist = [(dtrain,'train'),(dval,'val')]
model = xgb.train(plst,dtrain,num_rounds,watchlist,early_stopping_rounds=10,verbose_eval=50)
val_pred = model.predict(dval)
print(val_pred[len(val_pred)-30:])

[0]	train-rmse:58.3515	val-rmse:58.0864
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 10 rounds.
[50]	train-rmse:3.11512	val-rmse:3.13212
[99]	train-rmse:2.49091	val-rmse:2.54293
[ 61.34294891  61.83780289  62.78824234  62.12795258  62.12795258
  62.12795258  62.12795258  61.83780289  61.83780289  61.83780289
  62.05159378  62.34737396  62.11045837  61.81467819  61.6008873
  62.26117706  61.89103699  61.89103699  62.55132675  62.55132675
  62.55132675  62.79685211  64.02587891  65.39723969  65.83670807
  66.43056488  66.7399292   67.28089142  67.28089142  67.24097443]


In [10]:
plst = list(best_params.items())
watchlist = [(dtrain,'train')]
num_rounds = 100
dtrain = xgb.DMatrix(X,label=y)
model = xgb.train(plst,dtrain,num_rounds,watchlist,early_stopping_rounds=10,verbose_eval=10)

[0]	train-rmse:58.3638
Will train until train-rmse hasn't improved in 10 rounds.
[10]	train-rmse:20.9929
[20]	train-rmse:8.45534
[30]	train-rmse:4.66618
[40]	train-rmse:3.62201
[50]	train-rmse:3.24501
[60]	train-rmse:2.99566
[70]	train-rmse:2.85772
[80]	train-rmse:2.71144
[90]	train-rmse:2.61033
[99]	train-rmse:2.50582


In [11]:
hidden_x = pd.DataFrame({
    "day_1": get_timespan(df,1,145,0),
    "mean_3": get_timespan(df,3,145,0),
    "mean_7": get_timespan(df,7,145,0),
    "mean_14": get_timespan(df,14,145,0),
    "mean_30": get_timespan(df,30,145,0),
    "mean_60": get_timespan(df,60,145,0),
    "mean_140": get_timespan(df,140,145,0)})
dtest = xgb.DMatrix(hidden_x)
val_test=model.predict(dtest)
print(val_test[len(val_test)-30:])

[ 59.81932831  60.0846138   60.0846138   60.0846138   60.0846138
  60.17110443  60.49689484  60.63085556  60.63085556  60.63085556
  60.63085556  60.63085556  60.98329926  60.98329926  60.98329926
  60.98329926  60.98329926  60.98329926  60.98329926  60.98329926
  60.98329926  60.98329926  60.98329926  60.98329926  60.98329926
  60.98329926  60.98329926  60.98329926  60.98329926  61.56468964]
